In [1]:
from models.base import Base
from sqlalchemy.orm import backref, column_property, relationship, synonym, validates, sessionmaker
from sqlalchemy import Boolean
from sqlalchemy_utils.functions import drop_database, create_database

In [2]:
import sqlalchemy
from sqlalchemy import create_engine, ForeignKey
url = 'postgresql://admin:password@db:5432/test'
drop_database(url)



In [3]:
create_database(url)
engine = create_engine(url, echo=True)

In [4]:
from sqlalchemy import Column, Integer, String

In [5]:
# with cascade delete on identifier
#deleting the identifier does nothing with cascae delete on Identifier user relation
#deleting the user does not cascade to its related identifiers

In [6]:
# with cascade delete on user - deleting identifier does nothing
# deleting user deletes the identifiers

In [7]:
class Identifier(Base):
  __tablename__ = 'ids'

  id = Column(Integer, primary_key=True)
  value = Column(String)
  verified = Column(Boolean, default=False, nullable=False)
  user_uid = Column(Integer, ForeignKey('users.id'))
  user = relationship('User', foreign_keys=[user_uid], backref='id2')

  def verify(self):
    self.verified = True

In [8]:
class User(Base):
  __tablename__ = 'users'

  id = Column(Integer, primary_key=True)
  name = Column(String)
  identifiers = relationship('Identifier', foreign_keys=[Identifier.user_uid], cascade='save-update, merge, delete')

  def verify(self):
    for i in self.identifiers:
      i.verified = True

In [9]:
Session = sessionmaker(binds={
  Identifier:engine,
  User:engine,})
session = Session()

In [10]:
Base.metadata.create_all(bind=engine)



2021-05-22 17:58:44,030 INFO sqlalchemy.engine.base.Engine select version()


2021-05-22 17:58:44,031 INFO sqlalchemy.engine.base.Engine {}


2021-05-22 17:58:44,032 INFO sqlalchemy.engine.base.Engine select current_schema()


2021-05-22 17:58:44,032 INFO sqlalchemy.engine.base.Engine {}


2021-05-22 17:58:44,034 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1


...

2021-05-22 17:58:44,282 INFO sqlalchemy.engine.base.Engine {}


2021-05-22 17:58:44,283 INFO sqlalchemy.engine.base.Engine COMMIT


2021-05-22 17:58:44,286 INFO sqlalchemy.engine.base.Engine ALTER TABLE personal_identifiers ADD FOREIGN KEY(account_uuid) REFERENCES accounts (uuid)


2021-05-22 17:58:44,287 INFO sqlalchemy.engine.base.Engine {}


2021-05-22 17:58:44,288 INFO sqlalchemy.engine.base.Engine COMMIT


In [11]:
u = User(name="tuck")

In [12]:
i = Identifier(value="tucker@gmailc.om")

In [13]:
i.user

In [14]:
session.add(u)
session.commit()

2021-05-22 17:58:46,751 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2021-05-22 17:58:46,753 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name) VALUES (%(name)s) RETURNING users.id


2021-05-22 17:58:46,754 INFO sqlalchemy.engine.base.Engine {'name': 'tuck'}


2021-05-22 17:58:46,757 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
session.add(i)
session.commit()

2021-05-22 17:58:48,891 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2021-05-22 17:58:48,892 INFO sqlalchemy.engine.base.Engine INSERT INTO ids (value, verified, user_uid) VALUES (%(value)s, %(verified)s, %(user_uid)s) RETURNING ids.id


2021-05-22 17:58:48,892 INFO sqlalchemy.engine.base.Engine {'value': 'tucker@gmailc.om', 'verified': False, 'user_uid': None}


2021-05-22 17:58:48,894 INFO sqlalchemy.engine.base.Engine COMMIT


In [19]:
i.verified

False

In [20]:
i.verify()

In [21]:
i.verified

True

In [23]:
session.commit()

2021-05-22 17:57:32,219 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
session.query(Identifier).first().verified

2021-05-22 17:57:38,672 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.verified AS ids_verified, ids.user_uid AS ids_user_uid 
FROM ids 
 LIMIT %(param_1)s


2021-05-22 17:57:38,675 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


True

In [16]:
i.user_uid = u.id

2021-05-22 17:58:55,009 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2021-05-22 17:58:55,010 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name 
FROM users 
WHERE users.id = %(param_1)s


2021-05-22 17:58:55,010 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


In [17]:
i2 = Identifier(value="tucker2@gmail.com")

In [18]:
i.user

In [19]:
i2.user_uid = u.id

In [20]:
session.add(i)

In [21]:
session.add(i2)

In [22]:
session.commit()

2021-05-22 17:59:00,377 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.verified AS ids_verified 
FROM ids 
WHERE ids.id = %(param_1)s


2021-05-22 17:59:00,380 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


2021-05-22 17:59:00,387 INFO sqlalchemy.engine.base.Engine UPDATE ids SET user_uid=%(user_uid)s WHERE ids.id = %(ids_id)s


2021-05-22 17:59:00,389 INFO sqlalchemy.engine.base.Engine {'user_uid': 1, 'ids_id': 1}


2021-05-22 17:59:00,392 INFO sqlalchemy.engine.base.Engine INSERT INTO ids (value, verified, user_uid) VALUES (%(value)s, %(verified)s, %(user_uid)s) RETURNING ids.id


2021-05-22 17:59:00,393 INFO sqlalchemy.engine.base.Engine {'value': 'tucker2@gmail.com', 'verified': False, 'user_uid': 1}


2021-05-22 17:59:00,395 INFO sqlalchemy.engine.base.Engine COMMIT


In [23]:
u.identifiers[0].verif

2021-05-22 17:59:04,782 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2021-05-22 17:59:04,784 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name 
FROM users 
WHERE users.id = %(param_1)s


2021-05-22 17:59:04,784 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


2021-05-22 17:59:04,786 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.verified AS ids_verified, ids.user_uid AS ids_user_uid 
FROM ids 
WHERE %(param_1)s = ids.user_uid


2021-05-22 17:59:04,786 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


In [22]:
i.user

2021-05-20 22:05:41,953 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2021-05-20 22:05:41,957 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.user_uid AS ids_user_uid 
FROM ids 
WHERE ids.id = %(param_1)s


2021-05-20 22:05:41,960 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


2021-05-20 22:05:41,965 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name 
FROM users 
WHERE users.id = %(param_1)s


2021-05-20 22:05:41,967 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


In [23]:
i2.user

2021-05-20 22:05:42,201 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.user_uid AS ids_user_uid 
FROM ids 
WHERE ids.id = %(param_1)s


2021-05-20 22:05:42,202 INFO sqlalchemy.engine.base.Engine {'param_1': 2}


In [24]:
u.identifiers

2021-05-20 22:05:42,570 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.user_uid AS ids_user_uid 
FROM ids 
WHERE %(param_1)s = ids.user_uid


2021-05-20 22:05:42,571 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


In [25]:
u.id2

2021-05-20 22:05:42,984 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.user_uid AS ids_user_uid 
FROM ids 
WHERE %(param_1)s = ids.user_uid


2021-05-20 22:05:42,985 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


In [26]:
i.user_uid

1

In [27]:
i.user

In [28]:
session.flush()
session.commit()
session.flush()

2021-05-20 22:05:44,922 INFO sqlalchemy.engine.base.Engine COMMIT


In [30]:
session.query(User).delete()

2021-05-20 20:18:43,759 INFO sqlalchemy.engine.base.Engine DELETE FROM users


2021-05-20 20:18:43,760 INFO sqlalchemy.engine.base.Engine {}


IntegrityError: (psycopg2.errors.ForeignKeyViolation) update or delete on table "users" violates foreign key constraint "ids_user_uid_fkey" on table "ids"
DETAIL:  Key (id)=(1) is still referenced from table "ids".

[SQL: DELETE FROM users]
(Background on this error at: http://sqlalche.me/e/13/gkpj)

In [28]:
session.delete(i)

In [29]:

session.delete(u)




2021-05-20 22:05:50,582 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2021-05-20 22:05:50,584 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name 
FROM users 
WHERE users.id = %(param_1)s


2021-05-20 22:05:50,585 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


2021-05-20 22:05:50,586 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.user_uid AS ids_user_uid 
FROM ids 
WHERE %(param_1)s = ids.user_uid


2021-05-20 22:05:50,587 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


In [30]:
session.commit()

2021-05-20 22:05:51,404 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.user_uid AS ids_user_uid 
FROM ids 
WHERE %(param_1)s = ids.user_uid


2021-05-20 22:05:51,405 INFO sqlalchemy.engine.base.Engine {'param_1': 1}


2021-05-20 22:05:51,407 INFO sqlalchemy.engine.base.Engine UPDATE ids SET user_uid=%(user_uid)s WHERE ids.id = %(ids_id)s


2021-05-20 22:05:51,408 INFO sqlalchemy.engine.base.Engine ({'user_uid': None, 'ids_id': 1}, {'user_uid': None, 'ids_id': 2})


2021-05-20 22:05:51,409 INFO sqlalchemy.engine.base.Engine DELETE FROM ids WHERE ids.id = %(id)s


2021-05-20 22:05:51,409 INFO sqlalchemy.engine.base.Engine ({'id': 1}, {'id': 2})


2021-05-20 22:05:51,410 INFO sqlalchemy.engine.base.Engine DELETE FROM users WHERE users.id = %(id)s


2021-05-20 22:05:51,410 INFO sqlalchemy.engine.base.Engine {'id': 1}


2021-05-20 22:05:51,411 INFO sqlalchemy.engine.base.Engine COMMIT


In [31]:
r = session.query(Identifier).all()

2021-05-20 22:05:53,461 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)


2021-05-20 22:05:53,462 INFO sqlalchemy.engine.base.Engine SELECT ids.id AS ids_id, ids.value AS ids_value, ids.user_uid AS ids_user_uid 
FROM ids


2021-05-20 22:05:53,463 INFO sqlalchemy.engine.base.Engine {}


In [32]:
r

[]

In [33]:
session.query(User).all()

2021-05-20 22:06:01,198 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name 
FROM users


2021-05-20 22:06:01,199 INFO sqlalchemy.engine.base.Engine {}


[]

In [33]:
session.flush()

In [33]:
session.rollback()

2021-05-20 20:24:02,052 INFO sqlalchemy.engine.base.Engine ROLLBACK
